In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from matplotlib import pyplot as plt

В данной работе вы познакомитесь с простой нейронной сетью для решения задачи распознавания рукописных цифр из набора данных MNIST

![jupyter](https://storage.googleapis.com/tfds-data/visualization/fig/mnist-3.0.1.png)

### Создание нейронной сети

Создаем класс нейронной сети.

Наследуемся от nn.Module

Создаем методы init и forward.

В init инициализируем слои в поля класса (через self.название_слоя) см. пример. Пропишите сверточные слои, линейные слои, дропауты и maxpool

В forward создаем "путь" по которому пройдут входные данные. Функция должна вернуть результат всех преобразований.

#### Задание

1) Допишите в метод init необходимые слои из приложенной схемы

2) Допишите в метод forward "путь" входных данных через все слои и присвойте переменной output полученное значение

In [ ]:
%%html
<iframe src="https://drive.google.com/file/d/1M5ppV_EvwzG-265VbQb0ZsK6GL2wxMVC/preview" width="640" height="480" allow="autoplay"></iframe>

In [ ]:
class CN_Net(nn.Module):
    def __init__(self):
        super(CN_Net, self).__init__()
        # Пример слоя
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        """
        тут будут ваши слои
        """

    def forward(self, x):
        # Здесь преобразования входных данных представляют собой своеобразную матрешку
        # Глядя на схему "соберите" сеть

        # Пример использования слоя.
        x = self.conv1(x)
        x = F.relu(x)
        """
        тут будут ваши преобразования (слои, функции и тд)
        """
        output = ...
        return output


### Обучение и тестирование нейронной сети

Нейронная сеть представляет собой некоторую последовательность слоев, связанных между собой.
Их можно рассматривать как матричные преобразования.

Так как мы не знаем нужные веса нашей сети для решения поставленной задачи, то после каждого шага мы будем сверять полученные значения с целевыми, внося поправки в веса при помощи обратного распространения ошибки.

Создадим функцию для обучения нашей сети. На вход подаем:
- словарь, содержащий гиперпараметры
- модель
- устройство на котором будем считать (в нашем случае это CPU)
- загрузчик данных
- оптимизатор
- количество эпох

### Задание

Допишите нехватающие элементы в функции train

In [ ]:
def train(args: dict,
          model: nn.Module,
          device: torch.device,
          train_loader,
          optimizer: torch.optim,
          epoch: int):
    # переведем модель НС в режим обучения
    model.train()

    # в цикле пройдемся по загрузчику данных, получая из него батчи:
    # batch_idx - индекс отдельного батча
    # data - данные
    # target - целевая переменная
    for batch_idx, (data, target) in enumerate(train_loader):
        # перенесем данные на устройство
        data, target = data.to(device), target.to(device)

        # установить нулевые градиенты у оптимизатора
        ... # вместо многоточия ваш код

        # передать в модель данные и сохранить их в переменную output
        output = ... # вместо многоточия ваш код

        # посчитать функцию потерь между полученными результатами предсказания и целевыми показателями (используйте nll_loss)
        loss = ... # вместо многоточия ваш код

        # вызвать обратное распространение ошибки (у объекта loss)
        ... # вместо многоточия ваш код

        # сделать шаг оптимизатора
        ... # вместо многоточия ваш код

        # Выведем информацию о прогрессе обучения
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


Создадим функцию для тестирования нашей сети. На вход подаем:
- модель
- устройство, на котором будем считать
- загрузчик данных
- оптимизатор

### Задание

Допишите нехватающие элементы в функции test

In [ ]:
def test(model: nn.Module,
         device,
         test_loader):
    # переведем модель в режим предсказания
    model.eval()

    # создадим переменные для подсчета суммарного значения функции потерь и количества правильных ответов
    test_loss = 0.0
    correct = 0.0

    #  запустим цикл в режиме без расчета градиентов
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            # передадим данные в модель
            output = ... # вместо многоточия ваш код

            # просуммируем значения функции потерь на текущем шаге с общей
            # используйте ту же функцию потерь что и в train (чтобы получить значение вызовите метод .item())
            test_loss += ...  # вместо многоточия ваш код

            # найдем позицию (метку класса) максимального элемента предсказания при помощи argmax
            pred = ... # вместо многоточия ваш код

            # подсчитаем количество верных ответов
            correct += pred.eq(target.view_as(pred)).sum().item()

    # подсчитаем среднее значение функции потерь для всего набора данных
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss

### Гиперпараметры

Для обучения сети нам потребуется задать параметры от которых мы будем отталкиваться при формировании учебных наборов, шагов оптимизатора и т.д.

In [ ]:
args = {'batch_size': 64,
        'test_batch_size': 1000,
        'epochs': 14,
        'lr': 1.0,
        'gamma': 0.7,
        'log_interval': 1000,
        'save_model': False
        }

зададим устройство, на котором будем производить расчеты. Расчеты на GPU не будем рассматривать в рамках нашего курса. Если запускаете на collab то поменяйте параметр cpu на gpu

In [ ]:
device = torch.device("cpu")

создадим два словаря с ключами 'batch_size'
значения для них возьмем из args по ключам batch_size и test_batch_size соответственно

In [ ]:
train_kwargs = ... # вместо многоточия ваш код
test_kwargs = ... # вместо многоточия ваш код

создадим пайплайн предобработки данных из набора данных

Поместим в него преобразование в тензоры (для работы torch) и нормализацию данных

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

скачаем учебный и тестовый наборы данных при помощи datasets.MNIST

для учебного набора укажите:
- папку сохранения
- флаг скачивания учебного набора
- параметр transform=transform, который преобразует наши данные так, как мы задали выше

### Задание

Заполните аргументы в ячейке ниже для скачивания наборов данных

In [ ]:
dataset1 = datasets.MNIST(...) # вместо многоточия ваш код
dataset2 = datasets.MNIST(...) # вместо многоточия ваш код

Создадим загрузчики данных при помощи DataLoader из torch

In [ ]:
train_loader = DataLoader(dataset1, **train_kwargs)
test_loader = DataLoader(dataset2, **test_kwargs)

### Задание

инициализируйте модель:

In [ ]:
model = ...  # вместо многоточия ваш код

инициализируем оптимизатор и планировщик

Оптимизатор отвечает за стратегию нахождения минимума

Планировщик позволяет уменьшать шаг оптимизатора на заданную гамму

In [ ]:
optimizer = optim.Adadelta(model.parameters(), lr=args['lr'])
scheduler = StepLR(optimizer, step_size=1, gamma=args['gamma'])

Создадим цикл, в котором будет количество повторений равное количество эпох.
Получим из test значение лосс-функции на тестовой выборке.

Внутри тела цикла вызовем поочередно функции обучения и тестирования с требуемыми аргументами.

Насладимся результатами.

In [ ]:
losses = []
for epoch in range(1, args['epochs'] + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    loss = test(model, device, test_loader)
    losses.append(loss)

Отрисуем график функции потерь на тестовом наборе в зависимости от эпохи

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(losses)
plt.xlabel('Epochs', size=20)
plt.ylabel('Loss function', size=20)
plt.show()